In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import trimesh 
import numpy as np
import sys
import os
import inspect
from trimesh.exchange.binvox import voxelize_mesh
from trimesh import voxel as v
from trimesh.voxel import creation
import matplotlib.pyplot as plt
import math

from stl import mesh as stlmesh

from mpl_toolkits import mplot3d



from scipy.spatial import KDTree
from trimesh.ray import ray_pyembree
from pykdtree.kdtree import KDTree as pyKDTree

In [2]:
# parameter
N_KNN = 10  # number of edge from one sampled point
MAX_EDGE_LEN = 2.5  # [m] Maximum edge length
minDist = 0.1
maxDist = MAX_EDGE_LEN


pitch = 0.2
# 50,100,200,250,400,500
num_points = 1000   
file_obj = "test.obj"

class Edge:
    """
    Edge class for roadmap
    """
    def __init__(self,nodei,nodej,euclideanDist):
        self.start = [nodei.x,nodei.y,nodei.z]
        self.end = [nodej.x,nodej.y,nodej.z]
        self.edgeLength = euclideanDist

    def __str__(self):
        return str(self.start)+"---"+self.edgeLength+"---"+str(self.end)


class Node:
    """
    Node class for dijkstra search
    """

    def __init__(self, x, y, z, cost=0, parent_index=0):
        self.x = x
        self.y = y
        self.z = z
        self.cost = np.round(cost,3)
        self.parent_index = parent_index
        self.numEdges = 0
        self.neighbors = []
        self.id = 0
        self.gscore = self.cost
        self.fscore = math.inf

    def __str__(self):
        return str(self.x)+","+str(self.y)+","+str(self.z)#+","+str(self.cost) + "," + str(self.parent_index)
    def __eq__(self, other):
        if (isinstance(other, Node)):
            return self.x == other.x and self.y == other.y and self.z == other.z
        return false
    
    def set_numEdges(self,num):
        self.numEdges = num
    def get_numEdges(self):
        return self.numEdges
    def addNeighbors(self,neighbor):
        self.neighbors.append(neighbor)
    def setNodeIndex(self,idno):
        self.id = idno
    def getNodeIndex(self):
        return self.id
    def calcfscore(self,goal):
        self.fscore = self.cost+c2g(self,goal)
        self.h = c2g(self,goal)
    def getNeighbors(self):
        return self.neighbors
        
def c2g(current,goal):
    point1 = np.array((current.x,current.y,current.z))
    point2 = np.array((goal.x,goal.y,goal.z))
    return np.round(np.linalg.norm(point1 - point2),3)

def as_mesh(scene_or_mesh):
    """
    Convert a possible scene to a mesh.

    If conversion occurs, the returned mesh has only vertex and face data.
    """
    if isinstance(scene_or_mesh, trimesh.Scene):
        if len(scene_or_mesh.geometry) == 0:
            mesh = None  # empty scene
        else:
            # we lose texture information here
            mesh = trimesh.util.concatenate(
                tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
                    for g in scene_or_mesh.geometry.values()))
    else:
        assert(isinstance(mesh, trimesh.Trimesh))
        mesh = scene_or_mesh
    return mesh

def printVar(var):
    print(type(var))
    print(var)
    
def pop_node_lowest_fscore(queue):  # Priority Queue, outputs the node with least cost attached to it
    low = 0
    for i in range(len(queue)):
        if queue[i].fscore < queue[low].fscore:
            low = i
    return queue.pop(low)

def find_node(node, queue):
    for c in queue:
        if c==node:
            return queue.index(c)
        else:
            return None

In [3]:
scene = trimesh.Scene([])

voxeltomesh_scene = trimesh.Scene([])


sampled_scene = trimesh.Scene([])


point = np.asarray([[0.0,0.0,0.0]]) # right x , out z , up y
origin = trimesh.points.PointCloud(point)
col = np.asarray([0,0,0,255],dtype=np.uint8)
origin.vertices_color = col
scene.add_geometry(origin)





print("Loading",file_obj,"...")
mesh = trimesh.load(file_obj) # with plane to increase the bounding box
scene.add_geometry(mesh)


Loading test.obj ...


In [4]:
scene.show()

In [5]:
print("Mesh bounds: ",mesh.bounds)  
print("Min Bound: ",mesh.bounds[0])
print("Max Bound: ",mesh.bounds[1])
print("Mesh extent: ",mesh.extents)

print("Point Cloud of ",num_points," Sampled Points")
points = mesh.bounding_box_oriented.sample_volume(count=num_points)
# printVar(points)

startpoint = [0,0.5,-3.0] # blender y is -z
goalpoint = [0,10.0,0] # blender z is y

# MAKE UAV FLY TO START FROM SPAWN 

arr = np.asarray([startpoint,goalpoint])
print("Start conditions:")
print("Start point \t:",arr[0])
print("Goal point \t:",arr[1])

points = np.concatenate((arr,points))


index = (points / pitch).round().astype(int)
# pcd = trimesh.points.PointCloud(points[2:],colors=[[0,0,0,64] for i in points[2:]])
pcd = trimesh.points.PointCloud(points,colors=[[0,0,0,64] for i in points])
initialpoints = trimesh.points.PointCloud(points[0:2],colors=[[0,255,0,255] for i in points[0:2]])
scene.add_geometry([pcd,initialpoints])

Mesh bounds:  [[-7.09484  -0.791389 -8.      ]
 [ 8.342675 14.334061  8.      ]]
Min Bound:  [-7.09484  -0.791389 -8.      ]
Max Bound:  [ 8.342675 14.334061  8.      ]
Mesh extent:  [15.437515 15.12545  16.      ]
Point Cloud of  1000  Sampled Points
Start conditions:
Start point 	: [ 0.   0.5 -3. ]
Goal point 	: [ 0. 10.  0.]


['geometry_12', 'geometry_13']

In [6]:
createPCD = True
# if createPCD:
#     scene.add_geometry(pcd)

voxelize = True
if voxelize:
    mesh = as_mesh(mesh)
    print("Voxelizing with pitch (side length of cube): ", pitch)
    start_time = time.time()
    mesh_voxels = creation.voxelize(mesh, pitch=pitch, method='subdivide')
    mesh_voxels = v.VoxelGrid(mesh_voxels.encoding.dense,mesh_voxels.transform)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Finished Voxelizing...")
    mesh_voxels.fill(method='holes')
    scene.add_geometry(mesh_voxels.as_boxes(colors=(0, 0, 1, 0.3)))
 
    inpoints = []
    outpoints = []
    for i in pcd:
        if mesh_voxels.is_filled(i):
            # print("Filled")
            inpoints.append(i)
        else:
            # print("NOT Filled")
            outpoints.append(i)
    if createPCD:
        pcd1 = trimesh.points.PointCloud(inpoints,colors=[[255,0,0,64] for i in inpoints])
        scene.add_geometry(pcd1)
        voxeltomesh_scene.add_geometry(pcd1)
        pcd2 = trimesh.points.PointCloud(outpoints,colors=[[0,0,0,64] for i in outpoints])
        scene.add_geometry(pcd2)
        voxeltomesh_scene.add_geometry([pcd2,initialpoints])
        sampled_scene.add_geometry([pcd2,initialpoints])
        
    print("Outside voxels\t:",len(outpoints))
    print("Inside voxels\t:",len(inpoints))

    
mesh_voxels_center_points = mesh_voxels.points
voxelizedMesh = trimesh.voxel.ops.multibox(mesh_voxels_center_points, pitch=pitch, colors=(0,0,255,64))
voxeltomesh_scene.add_geometry(voxelizedMesh)


Voxelizing with pitch (side length of cube):  0.2
--- 14.27497124671936 seconds ---
Finished Voxelizing...
Outside voxels	: 839
Inside voxels	: 163


'geometry_3'

# Run

In [7]:
# PRM begins here
nodes = []
# nodes.append(startnode)
# nodes.append(goalnode)

edges = []
road_map = []
n_sample = len(points)
sample_x = points[:,0]
sample_y = points[:,1]
sample_z = points[:,2]

innodes = []
outnodes = []

for i in range(n_sample):
    node = Node(sample_x[i],sample_y[i],sample_z[i])
    if node:
        if mesh_voxels.is_filled(points[i]):
            innodes.append(node)
        else:
            outnodes.append(node)
            nodes.append(node)

In [8]:
print("Generating all possible free paths with MAX_EDGE_LEN",MAX_EDGE_LEN,"...")
start_time = time.time()
edgeCountMax = 0
for i in range(len(nodes)):
    print("Parsing through node",i)
    edgeCount = 0
    nodeiedges = []
    for j in range(i,len(nodes)):
        # while(edgeCount<=N_KNN):
            # print(edgeCount)
            diff_x = nodes[i].x - nodes[j].x
            diff_y = nodes[i].y - nodes[j].y
            diff_z = nodes[i].z - nodes[j].z
            euclideanDist = math.sqrt(diff_x**2 + diff_y**2 + diff_z**2)
            if euclideanDist >= minDist and euclideanDist <= maxDist:
                ray_origins = np.array([[nodes[i].x,nodes[i].y,nodes[i].z]])
                ray_directions = np.array([[-diff_x,-diff_y,-diff_z]])
                locations, index_ray, index_tri = voxelizedMesh.ray.intersects_location(ray_origins=ray_origins,ray_directions=ray_directions)
                #if points along edge not inside voxelizedMesh:
                if len(locations)==0:
                    # print(nodes[i],"  ",nodes[j],"\tN")
                    possibleEdge = Edge(nodes[i],nodes[j],euclideanDist)
                    nodes[i].addNeighbors(nodes[j])
                    edges.append(possibleEdge)
                    nodeiedges.append(possibleEdge)
                    nodes[i].set_numEdges(len(nodeiedges))
                    # print(i,nodes[i].get_numEdges())
                    # edgeCount+=1
                    if(len(nodeiedges)>=N_KNN):
                        edgeCountMax = len(nodeiedges)
                        break
                    
print("Max Edges from a possible node",edgeCountMax)
print("--- %s seconds ---" % (time.time() - start_time))
print("Finished creating",len(edges),"...")            
print("Total edges possible =",len(edges))

print("Joining",len(edges),"edges for visualization...")
for edge in edges:
    edge.start
    segments = np.vstack((edge.start,edge.end))
    path = trimesh.load_path(segments, process=False)
    # col = np.asarray([0,255,0,128],dtype=np.uint8)
    # path.vertices_color = col
    # printVar(path)
    trimesh.visual.color.ColorVisuals(mesh=path, vertex_colors=[255,0,0,255])
    voxeltomesh_scene.add_geometry(path)
    # ax.plot3D([edge.start[0],edge.end[0]], [edge.start[1],edge.end[1]], [edge.start[2],edge.end[2]], 'red')

# for i in range(0,len(arr)):
#     print(i,nodes[i],nodes[i].get_numEdges())


# your_mesh = stlmesh.Mesh.from_file('test.stl')
# ax.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))
# # Auto scale to the mesh size
# # scale = your_mesh.points.flatten(-1)
# # ax.auto_scale_xyz(scale, scale, scale)
# ax.view_init(-90, 90)

# ax.set_xlabel('x')
# ax.set_ylabel('y')
# ax.set_zlabel('z')
# plt.show()





Generating all possible free paths with MAX_EDGE_LEN 2.5 ...
Parsing through node 0 ...
Parsing through node 1 ...
Parsing through node 2 ...
Parsing through node 3 ...
Parsing through node 4 ...
Parsing through node 5 ...
Parsing through node 6 ...
Parsing through node 7 ...
Parsing through node 8 ...
Parsing through node 9 ...
Parsing through node 10 ...
Parsing through node 11 ...
Parsing through node 12 ...
Parsing through node 13 ...
Parsing through node 14 ...
Parsing through node 15 ...
Parsing through node 16 ...
Parsing through node 17 ...
Parsing through node 18 ...
Parsing through node 19 ...
Parsing through node 20 ...
Parsing through node 21 ...
Parsing through node 22 ...
Parsing through node 23 ...
Parsing through node 24 ...
Parsing through node 25 ...
Parsing through node 26 ...
Parsing through node 27 ...
Parsing through node 28 ...
Parsing through node 29 ...
Parsing through node 30 ...
Parsing through node 31 ...
Parsing through node 32 ...
Parsing through node 33 .

Parsing through node 287 ...
Parsing through node 288 ...
Parsing through node 289 ...
Parsing through node 290 ...
Parsing through node 291 ...
Parsing through node 292 ...
Parsing through node 293 ...
Parsing through node 294 ...
Parsing through node 295 ...
Parsing through node 296 ...
Parsing through node 297 ...
Parsing through node 298 ...
Parsing through node 299 ...
Parsing through node 300 ...
Parsing through node 301 ...
Parsing through node 302 ...
Parsing through node 303 ...
Parsing through node 304 ...
Parsing through node 305 ...
Parsing through node 306 ...
Parsing through node 307 ...
Parsing through node 308 ...
Parsing through node 309 ...
Parsing through node 310 ...
Parsing through node 311 ...
Parsing through node 312 ...
Parsing through node 313 ...
Parsing through node 314 ...
Parsing through node 315 ...
Parsing through node 316 ...
Parsing through node 317 ...
Parsing through node 318 ...
Parsing through node 319 ...
Parsing through node 320 ...
Parsing throug

Parsing through node 572 ...
Parsing through node 573 ...
Parsing through node 574 ...
Parsing through node 575 ...
Parsing through node 576 ...
Parsing through node 577 ...
Parsing through node 578 ...
Parsing through node 579 ...
Parsing through node 580 ...
Parsing through node 581 ...
Parsing through node 582 ...
Parsing through node 583 ...
Parsing through node 584 ...
Parsing through node 585 ...
Parsing through node 586 ...
Parsing through node 587 ...
Parsing through node 588 ...
Parsing through node 589 ...
Parsing through node 590 ...
Parsing through node 591 ...
Parsing through node 592 ...
Parsing through node 593 ...
Parsing through node 594 ...
Parsing through node 595 ...
Parsing through node 596 ...
Parsing through node 597 ...
Parsing through node 598 ...
Parsing through node 599 ...
Parsing through node 600 ...
Parsing through node 601 ...
Parsing through node 602 ...
Parsing through node 603 ...
Parsing through node 604 ...
Parsing through node 605 ...
Parsing throug

In [9]:
startSphere = trimesh.primitives.Sphere(radius=0.2,center=startpoint,subdivisions=2)
goalSphere = trimesh.primitives.Sphere(radius=0.2,center=goalpoint,subdivisions=2)
startSphere.visual.face_colors = np.asarray([0,255,0,255],dtype=np.uint8)
goalSphere.visual.face_colors = np.asarray([255,0,0,255],dtype=np.uint8)
visualization_scene = voxeltomesh_scene.copy()
visualization_scene.add_geometry([startSphere,goalSphere])

['geometry_2847', 'geometry_2848']

# Goal Point Visualization

# Start Point Visualization

# #Edges connected to Start & Goal

In [10]:
for i in range(len(nodes)):
    print(nodes[i],len(nodes[i].neighbors))
    if(i==1):
        break

0.0,0.5,-3.0 2
0.0,10.0,0.0 2


In [11]:
for i in range(len(nodes)):
    nodes[i].setNodeIndex(i)
#     print(nodes[i].getNodeIndex())

# A* Algorithm on Roadmap

In [13]:
open_set, closed_set = dict(), dict()
visited = []
start_node = nodes[0]
goal_node = nodes[1]

open_set[start_node.getNodeIndex()] = start_node

queue = []
queue.append(start_node)
while queue:
    current = pop_node_lowest_fscore(queue)
    visited.append(current.getNodeIndex())
    print(visited)
    checkgoal = current==goal_node
    if checkgoal:
        print("DONE.")
        print("Closest to goal: ",current)
        goal_node.parent = current
        goal_node.cost = current.cost+c2g(current,goal_node)
        print("Goal: \t", goal_node)
        print("Path cost: \t",goal_node.cost)
        
    neighbors = current.getNeighbors()
    for neighbor in neighbors:
        neighbor.parent = current
        neighbor.cost = c2g(current, neighbor)
        if(neighbor.getNodeIndex() not in visited):
            neighbor.fscore = neighbor.fscore + c2g(neighbor, goal_node)
            visited.append(neighbor.getNodeIndex())
            queue.append(neighbor)
        else:
            n_id = find_node(neighbor, queue)
            if n_id is not None:
                temp_node = queue[n_id]
                if temp_node.cost + c2g(temp_node, goal_node) > neighbor.cost + c2g(temp_node, goal_node):
                    temp_node.cost = neighbor.cost
                    temp_node.calcfscore(goal)
                    temp_node.parent = current
    

[0]
[0, 565, 743, 565]
[0, 565, 743, 565, 588, 677, 743]
[0, 565, 743, 565, 588, 677, 743, 747, 588]
[0, 565, 743, 565, 588, 677, 743, 747, 588, 666, 808, 810, 811, 677]
[0, 565, 743, 565, 588, 677, 743, 747, 588, 666, 808, 810, 811, 677, 747]
[0, 565, 743, 565, 588, 677, 743, 747, 588, 666, 808, 810, 811, 677, 747, 666]
[0, 565, 743, 565, 588, 677, 743, 747, 588, 666, 808, 810, 811, 677, 747, 666, 808]
[0, 565, 743, 565, 588, 677, 743, 747, 588, 666, 808, 810, 811, 677, 747, 666, 808, 810]
[0, 565, 743, 565, 588, 677, 743, 747, 588, 666, 808, 810, 811, 677, 747, 666, 808, 810, 811]
